In [1]:
import gc
from tqdm.notebook import tqdm
import scipy
import scipy.sparse

import os
import pandas as pd
import dask.dataframe as dd
import scipy.sparse
import pandas as pd
import numpy as np

import optuna
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
import pickle
import bz2
import shutil
from bz2 import BZ2Compressor
import random
import torch

import torch.nn as nn
import torch.utils.data as torch_data
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence

from file_utils import saver_state, loader_state, shuffle, compressed_read, compressed_write, decompress

from torch.utils.tensorboard import SummaryWriter

import shutil

In [ ]:
do_compression = False

In [5]:
#Delte old data if there is. Recreate data directories.

path = os.path.join("nn_data2","train")
if os.path.isdir(path):
    shutil.rmtree(path)
os.mkdir(os.path.join("nn_data2","train"))

path = os.path.join("nn_data2","dev")
if os.path.isdir(path):
    shutil.rmtree(path)
os.mkdir(os.path.join("nn_data2","dev"))


In [6]:
#copy data from read only directory to work directory
for i in range(12):
    shutil.copy(os.path.join("nn_data2","clear","chunk"+str(i)+".bz2"), os.path.join("nn_data2","train"))

In [7]:
#if not flag than do not perform compression due to speed reasons            
if(do_compression == False):
    decompress("train")

  0%|          | 0/12 [00:00<?, ?it/s]

In [9]:
shutil.copy(os.path.join("nn_data2","train","chunk"+str(10)+".bz2"), os.path.join("nn_data2","dev"))
shutil.copy(os.path.join("nn_data2","train","chunk"+str(11)+".bz2"), os.path.join("nn_data2","dev"))

'nn_data2\\dev\\chunk11.bz2'

In [10]:
shuffle()

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

In [11]:
#make classifier validation
with open('feat2id.pickle', 'rb') as handle:
    feat2id = pickle.load(handle)

with open('id2feat.pickle', 'rb') as handle:
    id2feat = pickle.load(handle)

with open('global_mapping.pickle', 'rb') as handle:
    global_mapping = pickle.load(handle)

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

In [12]:
pad_values = {}
for ft in feat2id.keys():
    MaxFeatureRange = len(global_mapping[ft])+1+1+1 #reserve space for special tokens
    pad_values[ft] = MaxFeatureRange

In [13]:
import string
latters = list(string.ascii_uppercase)+list(string.ascii_lowercase)
latters2id = {k:v for k,v in zip(latters, range(len(latters)))}
id2latters = {v:k for k,v in latters2id.items()}

#next example generator. Read files from chunk to chunk and yiel exaples from each chunk

def get_next_example(part = "train", avaliable_chunks=(0,1,2,3,4,5,6,7,8,9, 10, 11)):
    for chunk_i in avaliable_chunks:
        with open(os.path.join("nn_data2",part,"chunk"+str(chunk_i)+".bz2"), "rb") as f:
            lines = compressed_read(f, do_compression = do_compression)
        lines = lines.split('\n')
        for l in lines:
            data = l.split(' ')
            words = data[:-1]
            y = data[-1]
            
            list_of_tokens_lists = [[latters2id[latter] for latter in word] for word in words]
            
            list_of_ids_lists = []
            padding_shift = 3
            
            for tokl, nom in zip(list_of_tokens_lists, range(len(list_of_tokens_lists)) ):
                list_of_ids_lists.append([global_mapping[id2feat[nom]][x] + padding_shift for x in tokl])
            yield list_of_ids_lists + [int(y)]

In [14]:
max_pos_len = 50

In [15]:
#data convertion functions for classifier
def process_feature(l, max_pos_len):
    if(len(l) < max_pos_len):
        l = [1] + l  + [0]*(max_pos_len - len(l))
    elif(len(l) > max_pos_len):
        l = [1] + l[-max_pos_len:] #[::-1]
    else:
        l = [1] + l  #[::-1]
    return l

#gather examples into batch
def collate_fn(batch_list, max_pos_len):
    feat_lists = [[] for _ in range(len(id2feat))]
    y = []
    lens = []
    for example in batch_list:
        for i in range(len(id2feat)):
            l = example[i]
            if(i==0):
                lens.append(len(l) + 1 if(len(l) < max_pos_len) else max_pos_len+1)
            l  = process_feature(l, max_pos_len)
            feat_lists[i].append(l)
        y.append(example[-1])
    
    return ([torch.tensor(x, dtype=torch.int32) for x in feat_lists], torch.tensor(y, dtype=torch.int64), lens)

#generate batches for classifier
def batch_gen(batch_size, max_pos_len = 50, part = "train", avaliable_chunks=(0,1,2,3,4,5,6,7,8,9, 10, 11)):
    batch_list = []
    i = 0
    for x in get_next_example(part, avaliable_chunks):
        batch_list.append(x)
        i += 1
        if(i == batch_size):
            yield collate_fn(batch_list, max_pos_len)
            batch_list = []
            i = 0
    if(len(batch_list)>0):
        yield collate_fn(batch_list)


In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Using device:  cuda


In [19]:
def eval_dev_classifier(model, nbatch):
    targets = []
    preds = []
    ii = 0
    for X, y, lens in tqdm(batch_gen(1000, max_pos_len, "dev", (10,11))):
        for i in range(len(X)):
            X[i] = X[i].to(device)
            y[i] = y[i].to(device)
        tgt = y.numpy()
        output = model(X, lens)[:,1] 
        pr = output.detach().cpu().numpy()
        targets.append(tgt)
        preds.append(np.exp(pr))
        ii += 1
        
        if(ii>nbatch):
            break
    return roc_auc_score(np.hstack(targets), np.hstack(preds))

In [20]:
from torch.optim.lr_scheduler import ExponentialLR

In [21]:
def train_classifier(epoches_gone, experiment_name, device):
    load_dir = os.path.join(experiment_name, "lm_chekpoints", str(epoches_gone), "model.pt")
    model = loader_state(load_dir)
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    scheduler = ExponentialLR(optimizer, gamma = (1e-5/1e-3)**(1/(6000*7)))

    loss_function = torch.nn.NLLLoss(weight=torch.tensor([1., 0.97/0.03]).to(device))#0.97/0.03
    
    logdir=os.path.join(experiment_name, "classifier_logs", str(epoches_gone))
    
    if os.path.isdir(logdir):
        shutil.rmtree(logdir)

    os.mkdir(logdir)

    writer=SummaryWriter(log_dir=logdir)
    
    
    model.train()
    epoches=3
    ii = 0
    for ep in range(epoches):
        for X, y, lens in tqdm(batch_gen(500, max_pos_len)):
            for i in range(len(X)):
                X[i] = X[i].to(device)
            y = y.to(device)
            output = model(X, lens)
            loss = loss_function(output, y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            scheduler.step()
            writer.add_scalar('Loss/train', loss.item() , ii)
            if(ii%200 == 0):
                print(loss.item())
            if(ii%1000 == 0):
                model.eval()
                ev = eval_dev_classifier(model, 50)
                print("after",ii,"steps short auc dev", ev)
                writer.add_scalar('auc dev short', ev , ii)
                model.train()
            ii+=1
        #break
        model.eval()
        ev = eval_dev_classifier(model, 10**4)
        print("after",ep,"epoches full auc dev", ev)
        writer.add_scalar('auc dev long', ev , ii)
        model.train()
        shuffle("train")
    return model
    

In [22]:
experiment_name = 'with_test_nn2'

In [ ]:
#load state of the pretrained encoder and run classifier training
model = train_classifier(0, experiment_name, device)

In [26]:
import string
latters = list(string.ascii_uppercase)+list(string.ascii_lowercase)
latters2id = {k:v for k,v in zip(latters, range(len(latters)))}
id2latters = {v:k for k,v in latters2id.items()}

#test loader

def get_next_example_test(part = "test", avaliable_chunks=(0,1)):
    for chunk_i in avaliable_chunks:
        with open(os.path.join("nn_data2",part,"chunk"+str(chunk_i)+".bz2"), "rb") as f:
            lines = compressed_read(f, do_compression = True)
        lines = lines.split('\n')
        for l in lines:
            data = l.split(' ')
            words = data[:]
            
            list_of_tokens_lists = [[latters2id[latter] for latter in word] for word in words]
            
            list_of_ids_lists = []
            padding_shift = 3
            
            for tokl, nom in zip(list_of_tokens_lists, range(len(list_of_tokens_lists)) ):
                list_of_ids_lists.append([global_mapping[id2feat[nom]][x] + padding_shift for x in tokl])
            yield list_of_ids_lists

In [27]:
def process_feature(l, max_pos_len):
    if(len(l) < max_pos_len):
        l = [1] + l  + [0]*(max_pos_len - len(l))
    elif(len(l) > max_pos_len):
        l = [1] + l[-max_pos_len:] #[::-1]
    else:
        l = [1] + l  #[::-1]
    return l

In [28]:
def collate_fn_test(batch_list, max_pos_len):
    #max_pos_len = 50
    feat_lists = [[] for _ in range(len(id2feat))]
    y = []
    lens = []
    for example in batch_list:
        for i in range(len(id2feat)):
            l = example[i]
            if(i==0):
                lens.append(len(l) + 1 if(len(l) < max_pos_len) else max_pos_len+1)
            
            l  = process_feature(l, max_pos_len)
            
            feat_lists[i].append(l)
    return ([torch.tensor(x, dtype=torch.int32) for x in feat_lists], lens)

In [29]:
def batch_gen_test(batch_size, max_pos_len):
    batch_list = []
    i = 0
    for x in get_next_example_test():
        batch_list.append(x)
        i += 1
        if(i == batch_size):
            yield collate_fn_test(batch_list, max_pos_len)
            batch_list = []
            i = 0
    if(len(batch_list)>0):
        yield collate_fn(batch_list)

In [ ]:
#predict test
preds = []
model.eval()
for X, lens in tqdm(batch_gen_test(1000, max_pos_len)):
    for i in range(len(X)):
        X[i] = X[i].to(device)
    output = model(X, lens)[:,1] 
    pr = output.detach().cpu().numpy()
    preds.append(np.exp(pr))
    
preds = np.hstack(preds)

In [31]:
#save preds
df = pd.DataFrame({
        "id": list(range(3000000, 3500000)),
        "score": preds
    })

In [32]:
df.to_csv("submission.csv", index=None)

Participants were offered to predict the debtor's default by his loan portfolio. Loan history was given for each bank's debtors. There is information about something around 5000000 debtors for the last 2 years. The competition's data was strongly anonymized. Continuous features were transformed into categorical through histogram splitting. As a result, each feature has no more than 30 values. Data have no explicit information about time, but debt information was sorted by time. The information in the test sample is in time after the information in the training sample.

I was developing lgbm and lstm-based rnn models. In order to encode historical data to lgbm, I calculated the probability of meeting this loan in the portfolio of a bankrupt debtor for each loan. The meta-features were calculated by the out-of-fold stacking method. We can perceive these features as an assessment of how risky a loan is. That features combined with count-aggregated features yield lgbm's quality comparable with rnn models.

Let's see how some count-aggregation features change their values over time.

As we can see dataset endures a strong "dataset shift". There is a certain amount of resample and reweight classical methods to deal with that problem, but usually, they work only for weak models. No surprise, that they didn't work for my lgbm and nn models. Polynomial decaying weights somewhat improved lgbm's quality but not nn's.

Further investigation showed that we could consider the "Dataset shift" problem as a special case of the domain adaptation problem. For nn models, there are a certain amount of advanced methods like ADDA and DANN. But usually, they work well for CNN, and they didn't work for me. For LSTM there are classical domain adaptation methods like LM and Autoencoder pretraining. Autoencoder pretraining showed a much stronger result than LM pretraining. And also its greatly outperformed lgbm with weight decay. As a result my final model was RNN-baseline with pretraining:).